# LAB3 - N-Puzzle

In [3]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

### INITIALIZATION

In [4]:
PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])

In [5]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(index[0]) for index in np.where(state == 0)]#return the position of the empty tile
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [7]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 382663.69it/s]


array([[17,  1,  3, 23,  9],
       [ 5,  7,  2, 10,  4],
       [11, 22, 19, 20, 18],
       [ 8, 21, 24, 15,  6],
       [ 0, 16, 13, 14, 12]])